In [16]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout

import os
import datetime
%load_ext tensorboard
from skimage import color, exposure
from sklearn.metrics import accuracy_score
from hyperopt import hp, STATUS_OK, tpe, Trials,fmin

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [2]:
cd '/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/'

/content/drive/My Drive/Colab Notebooks/matrix/matrix_three


In [3]:
cd dw_matrix_road_sign/

/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign


In [4]:
ls

data/  day2_matrix3.ipynb  Day3_matrix3.ipynb  LICENSE  logs/  README.md


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
df=pd.read_csv('data/signnames.csv')
labels_dict=df.to_dict()['b']

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)
input_shape=X_train.shape[1:]
num_classes = y_train.shape[1]

In [8]:
test['features'][1].shape

(32, 32, 3)

In [0]:
def train_model(model, X_train, y_train,params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer = 'Adam', metrics=['accuracy'])
    
  logdir= os.path.join("logs", datetime.datetime.now().strftime("Y%m%d-%D%M%S")) #siceżka do katlogu z logami
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1) 

  model.fit(
      X_train, 
      y_train,
      batch_size=params_fit.get('batch_size', 128), #dzielenie danych na paczki 
      epochs=params_fit.get('epochs', 5),
      verbose= params_fit.get('verbose', 1), #wypisanie stanu
      validation_data=params_fit.get('validation_data', (X_train,y_train)),
      callbacks=[tensorboard_callback] 
  )

  return model

In [0]:
def predict(model_trained, X_test, y_test, scoring=accuracy_score):

  y_test_norm = np.argmax(y_test, axis=1)

  y_pred_prob = model_trained.predict(X_test)
  y_pred= np.argmax(y_pred_prob, axis=1) #np.argmaxindex max wartości

  return accuracy_score(y_test_norm, y_pred)

In [0]:
def gen_cnn_v5(input_shape, num_classes):
  return  Sequential([
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape,padding='same'),
  MaxPool2D(),
  Dropout(0.3), #dropout zpobiega przseuczaniu
  

  Conv2D(filters=64, kernel_size=(3,3), activation='relu',padding='same'),
  Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(0.3),

  Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(0.3),

  Flatten(),

  Dense(1024, activation='relu'),
  Dropout(0.3),

  Dense(1024, activation='relu'),
  Dropout(0.3),

  Dense(num_classes, activation='softmax'),
  ])

#gen_cnn_v4(input_shape, num_classes).summary()

In [13]:
model = gen_cnn_v5(input_shape, num_classes)
model_trained=train_model(model, X_train, y_train, params_fit={})
predict(model_trained, X_test,y_test)

Epoch 1/5
272/272 [==============================] - 5s 17ms/step - loss: 2.2655 - accuracy: 0.3556 - val_loss: 0.8588 - val_accuracy: 0.7366
Epoch 2/5
272/272 [==============================] - 4s 16ms/step - loss: 0.7199 - accuracy: 0.7704 - val_loss: 0.1730 - val_accuracy: 0.9498
Epoch 3/5
272/272 [==============================] - 4s 16ms/step - loss: 0.3397 - accuracy: 0.8963 - val_loss: 0.0725 - val_accuracy: 0.9790
Epoch 4/5
272/272 [==============================] - 4s 15ms/step - loss: 0.2263 - accuracy: 0.9328 - val_loss: 0.0421 - val_accuracy: 0.9885
Epoch 5/5
272/272 [==============================] - 4s 15ms/step - loss: 0.1747 - accuracy: 0.9477 - val_loss: 0.0314 - val_accuracy: 0.9911


0.9691609977324263

In [0]:
def func_obj(params):
  model=gen_cnn_v5(input_shape, num_classes)
  model.compile(loss='categorical_crossentropy', optimizer = 'Adam', metrics=['accuracy'])
    
  model.fit(
      X_train, 
      y_train,
      batch_size=int(params.get('batch_size', 128)), #dzielenie danych na paczki 
      epochs=7,
      verbose= 0, #wypisanie stanu
  )
  accuracy=model.evaluate(X_test, y_test)[0]
  print(params, 'accuracy={}'.format(accuracy))
  return {'loss': -accuracy, 'status':STATUS_OK, 'model':model}

In [24]:
space= {
    'batch_size':hp.quniform('batch_size', 50, 200, 20)
}

best=fmin(
    func_obj,
    space,
    tpe.suggest,
    1,
    Trials()
)

  1/138 [..............................]
 - ETA: 0s - loss: 0.3448 - accuracy: 0.8438

 11/138 [=>............................]
 - ETA: 0s - loss: 0.1067 - accuracy: 0.9631

 19/138 [===>..........................]
 - ETA: 0s - loss: 0.0961 - accuracy: 0.9688

 26/138 [====>.........................]
 - ETA: 0s - loss: 0.2667 - accuracy: 0.9303

 34/138 [======>.......................]
 - ETA: 0s - loss: 0.2079 - accuracy: 0.9449

 41/138 [=======>......................]
 - ETA: 0s - loss: 0.1798 - accuracy: 0.9520


In [0]:
def get_model(params):
  return  Sequential([
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
  MaxPool2D(),
  Dropout(params['dropout_cnn_block_one']), #dropout zpobiega przseuczaniu
  

  Conv2D(filters=64, kernel_size=(3,3), activation='relu',padding='same'),
  Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(params['dropout_cnn_block_two']),

  Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(params['dropout_cnn_block_three']),

  Flatten(),

  Dense(1024, activation='relu'),
  Dropout(params['dropout_dense_block_one']),

  Dense(1024, activation='relu'),
  Dropout(params['dropout_dense_block_two']),

  Dense(num_classes, activation='softmax'),
  ])

In [0]:
def func_obj_2(params):
  model=get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer = 'Adam', metrics=['accuracy'])
    
  model.fit(
      X_train,
      y_train,
      batch_size=int(params.get('batch_size', 128)), #dzielenie danych na paczki 
      epochs=5,
      verbose= 0, #wypisanie stanu
  )
  score= model.evaluate(X_test, y_test, verbose =0)
  accuracy=score[1]
  print(params, 'accuracy={}'.format(accuracy))
  return {'loss': -accuracy, 'status':STATUS_OK, 'model':model}

In [51]:
space2= {
    'batch_size':hp.quniform('batch_size', 50, 200, 20),
    'dropout_cnn_block_one':hp.quniform('dropout_cnn_block_one', 0.3, 0.5, 0.1),
    'dropout_cnn_block_two':hp.quniform('dropout_cnn_block_two', 0.3, 0.5, 0.1),
    'dropout_cnn_block_three':hp.quniform('dropout_cnn_block_three', 0.3, 0.5, 0.1),

    'dropout_dense_block_one':hp.quniform('dropout_dense_block_one', 0.3, 0.7, 0.1),
    'dropout_dense_block_two':hp.quniform('dropout_dense_block_two', 0.3, 0.7, 0.1),
}

best=fmin(
    func_obj_2,
    space2,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 60.0, 'dropout_cnn_block_one': 0.4, 'dropout_cnn_block_three': 0.4, 'dropout_cnn_block_two': 0.30000000000000004, 'dropout_dense_block_one': 0.4, 'dropout_dense_block_two': 0.5}
accuracy=0.956462562084198
{'batch_size': 200.0, 'dropout_cnn_block_one': 0.4, 'dropout_cnn_block_three': 0.4, 'dropout_cnn_block_two': 0.4, 'dropout_dense_block_one': 0.5, 'dropout_dense_block_two': 0.4}
accuracy=0.938548743724823
{'batch_size': 100.0, 'dropout_cnn_block_one': 0.30000000000000004, 'dropout_cnn_block_three': 0.4, 'dropout_cnn_block_two': 0.30000000000000004, 'dropout_dense_block_one': 0.6000000000000001, 'dropout_dense_block_two': 0.7000000000000001}
accuracy=0.8795918226242065
{'batch_size': 80.0, 'dropout_cnn_block_one': 0.4, 'dropout_cnn_block_three': 0.4, 'dropout_cnn_block_two': 0.5, 'dropout_dense_block_one': 0.5, 'dropout_dense_block_two': 0.4}
accuracy=0.9387755393981934
{'batch_size': 80.0, 'dropout_cnn_block_one': 0.30000000000000004, 'dropout_cnn_block_three': 0.4, 'dr

In [52]:
best

{'batch_size': 80.0,
 'dropout_cnn_block_one': 0.30000000000000004,
 'dropout_cnn_block_three': 0.30000000000000004,
 'dropout_cnn_block_two': 0.30000000000000004,
 'dropout_dense_block_one': 0.4,
 'dropout_dense_block_two': 0.6000000000000001}